In [1]:
import os
import sys
from pathlib import Path

import numpy as np
import pandas as pd
from sklearn.metrics import accuracy_score

sys.path.append("..")
from otc.metrics.metrics import effective_spread

import wandb

from tqdm.auto import tqdm

from scipy.stats import wilcoxon
import warnings


/pfs/data5/home/kit/stud/uloak/thesis/thesis/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# set here globally
exchange = "cboe"
models = ["gbm", "classical"] # ["gbm", "classical"]
subset = "test" # "all"
strategy = "supervised"  # "transfer"


In [3]:
key = f"{exchange}_{strategy}_{subset}"
dataset = f"fbv/thesis/{exchange}_{strategy}_raw:latest"

os.environ["GCLOUD_PROJECT"] = "flowing-mantis-239216"

run = wandb.init(project="thesis", entity="fbv")

# load unscaled data
artifact = run.use_artifact(dataset)  # type: ignore
data_dir = artifact.download()

# load results
result_dirs = []
for model in models:
    results = f"fbv/thesis/{exchange}_{model}_{strategy}_{subset}:latest"
    artifact = run.use_artifact(results)  # type: ignore
    result_dir = artifact.download()
    result_dirs.append(result_dir)


Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: karelze (fbv). Use `wandb login --relogin` to force relogin


wandb: Downloading large artifact cboe_supervised_raw:latest, 1937.19MB. 3 files... 
wandb:   3 of 3 files downloaded.  
Done. 0:0:0.0
wandb: Downloading large artifact cboe_gbm_supervised_test:latest, 56.26MB. 1 files... 
wandb:   1 of 1 files downloaded.  
Done. 0:0:0.0
wandb: Downloading large artifact cboe_classical_supervised_test:latest, 212.93MB. 1 files... 
wandb:   1 of 1 files downloaded.  
Done. 0:0:0.0


In [4]:
# p. 35-38
columns = [
    "buy_sell",
    "EXPIRATION",
    "QUOTE_DATETIME",
    "TRADE_SIZE",
    "TRADE_PRICE",
    "ask_ex",
    "bid_ex",
    "myn",
    "OPTION_TYPE",
    "issue_type",
]


if subset == "all":
    train = pd.read_parquet(
        Path(data_dir, "train_set"), engine="fastparquet", columns=columns
    )
    val = pd.read_parquet(
        Path(data_dir, "val_set"), engine="fastparquet", columns=columns
    )
    test = pd.read_parquet(
        Path(data_dir, "test_set"), engine="fastparquet", columns=columns
    )
    eval_data = pd.concat([train, val, test])
    del train, val, test

elif subset == "test":
    eval_data = pd.read_parquet(
        Path(data_dir, "test_set"), engine="fastparquet", columns=columns
    )


results = []
for i, model in tqdm(enumerate(models)):
    result = pd.read_parquet(Path(result_dirs[i], "results"), engine="fastparquet")
    result.columns = pd.MultiIndex.from_product([[model], result.columns])
    results.append(result)

results_data = pd.concat(results, axis=1, names=models)

assert len(eval_data) == len(results_data)

X_print = eval_data

del results


2it [00:02,  1.26s/it]


In [5]:
# FIXME: select a subset of results for testing.
results_data = results_data[
    [
        ("gbm", "gbm(classical)"),
        ("gbm", "gbm(classical-size)"),
        ("gbm", "gbm(ml)"),
        ("classical", "tick(ex)"),
        ("classical", "quote(ex)"),
        ("classical", "lr(ex)"),
        ("classical", "emo(ex)"),
        ("classical", "clnv(ex)"),
        (
            "classical",
            "trade_size(ex)->quote(best)->depth(best)->quote(ex)->depth(ex)->rev_tick(all)",
        ),
    ]
]


In [6]:
results_data


gbm                             classical            \
         gbm(classical) gbm(classical-size) gbm(ml)  tick(ex) quote(ex)   
index                                                                     
29930521              1                   1       1       1.0      -1.0   
29894717              1                   1       1      -1.0       1.0   
29909660              1                   1       1      -1.0      -1.0   
29939322             -1                  -1      -1      -1.0      -1.0   
29883818             -1                   1       1      -1.0       1.0   
...                 ...                 ...     ...       ...       ...   
37155409              1                  -1      -1       1.0      -1.0   
37155410              1                   1       1       1.0       1.0   
37104659             -1                  -1      -1      -1.0       1.0   
37108365              1                   1       1      -1.0      -1.0   
37155411              1                   1       1      -1.0      -1.0   

                                  \
         lr(ex) emo(ex) clnv(ex)   
index                              
29930521   -1.0    -1.0     -1.0   
29894717    1.0     1.0      1.0   
29909660   -1.0    -1.0     -1.0   
29939322   -1.0     1.0     -1.0   
29883818    1.0     1.0      1.0   
...         ...     ...      ...   
37155409   -1.0    -1.0     -1.0   
37155410    1.0     1.0      1.0   
37104659    1.0     1.0      1.0   
37108365   -1.0    -1.0     -1.0   
37155411   -1.0    -1.0     -1.0   

                                                                                        
         trade_size(ex)->quote(best)->depth(best)->quote(ex)->depth(ex)->rev_tick(all)  
index                                                                                   
29930521                                                1.0                             
29894717                                                1.0                             
29909660                                               -1.0                             
29939322                                               -1.0                             
29883818                                                1.0                             
...                                                     ...                             
37155409                                               -1.0                             
37155410                                                1.0                             
37104659                                               -1.0                             
37108365                                                1.0                             
37155411                                                1.0                             

[7431766 rows x 9 columns]

### Robustness Checks

In [7]:
# prepare columns for printing
X_print["ttm"] = (
    X_print["EXPIRATION"].dt.to_period("M")
    - X_print["QUOTE_DATETIME"].dt.to_period("M")
).apply(lambda x: x.n)

X_print["year"] = X_print["QUOTE_DATETIME"].dt.year

bins_tradesize = [-1, 1, 3, 5, 11, np.inf]
trade_size_labels = ["(0,1]", "(1,3]", "(3,5]", "(5,11]", ">11"]
X_print["TRADE_SIZE_binned"] = pd.cut(
    X_print["TRADE_SIZE"], bins_tradesize, labels=trade_size_labels
)

# p. 38
bins_years = [2004, 2007, 2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017]
year_labels = [
    "2005-2007",
    "2008-2010",
    "2011",
    "2012",
    "2013",
    "2014",
    "2015",
    "2016",
    "2017",
]
X_print["year_binned"] = pd.cut(X_print["year"], bins_years, labels=year_labels)

# p. 37
bins_ttm = [-1, 1, 2, 3, 6, 12, np.inf]
ttm_labels = [
    "<= 1",
    "(1-2]",
    "(2-3]",
    "(3-6]",
    "(6-12]",
    "> 12",
]
X_print["ttm_binned"] = pd.cut(X_print["ttm"], bins_ttm, labels=ttm_labels)

# Security type
# see 3.0a-mb-explanatory-data-analysis.ipynb
X_print["issue_type"] = X_print["issue_type"].map(
    {
        "0": "Stock option",
        "A": "Index option",
        "7": "Others",
        "F": "Others",
        "%": "Others",
        " ": "Others",
    }
)

# Moneyness p. 38
bins_myn = [-1, 0.7, 0.9, 1.1, 1.3, np.inf]
myn_labels = [
    "<= 0.7",
    "(0.7-0.9]",
    "(0.9-1.1]",
    "(1.1-1.3]",
    "> 1.3",
]
X_print["myn_binned"] = pd.cut(X_print["myn"], bins_myn, labels=myn_labels)

# mid p. 31 + extra category for unknowns
ask = X_print["ask_ex"]
bid = X_print["bid_ex"]
trade_price = X_print["TRADE_PRICE"]

mid = np.where(ask >= bid, (ask + bid) * 0.5, np.nan)
half_spread = np.where(ask >= bid, (ask - bid) * 0.5, np.nan)

prox_quotes = np.where(
    trade_price == mid,
    0,  # at mid
    np.where(
        ((mid - half_spread) < trade_price) & (trade_price < (mid + half_spread)),
        1,  # inside
        np.where(
            (trade_price == ask) | (bid == trade_price),
            2,  # at quotes
            np.where(
                (trade_price < (mid - half_spread))
                | ((mid + half_spread) > trade_price),
                3,
                4,
            ),
        ),
    ),
)  # outside + unclassifiable

bins_prox = [-np.inf, 0, 1, 2, 3, 4]
prox_labels = [
    "at mid",
    "inside",
    "at quotes",
    "outside",
    "unknown",
]

X_print["prox_q_binned"] = pd.cut(prox_quotes, bins_prox, labels=prox_labels)
X_print["mid"] = mid

# clean up empty buckets, as it causes empty grouping in result set generation
X_print["year_binned"] = X_print["year_binned"].cat.remove_unused_categories()
X_print["myn_binned"] = X_print["myn_binned"].cat.remove_unused_categories()
X_print["ttm_binned"] = X_print["ttm_binned"].cat.remove_unused_categories()
X_print["prox_q_binned"] = X_print["prox_q_binned"].cat.remove_unused_categories()

X_print.drop(
    columns=[
        "EXPIRATION",
        "QUOTE_DATETIME",
        "TRADE_SIZE",
        "ttm",
        "myn",
        "ask_ex",
        "bid_ex",
        "year",
    ],
    inplace=True,
)


In [8]:
X_print = pd.concat([X_print, results_data], axis=1)


In [9]:
X_print.head().T


index,29930521,29894717,29909660,29939322,29883818
buy_sell,1,1,1,-1,1
TRADE_PRICE,0.02,0.27,0.2,0.12,1.85
OPTION_TYPE,C,P,C,C,C
issue_type,Others,Stock options,Stock options,Others,Stock options
TRADE_SIZE_binned,"(0,1]","(5,11]","(0,1]","(0,1]","(3,5]"
year_binned,2016,2016,2016,2016,2016
ttm_binned,<= 1,<= 1,(1-2],<= 1,(1-2]
myn_binned,(0.7-0.9],(0.9-1.1],(0.7-0.9],(0.9-1.1],(0.9-1.1]
prox_q_binned,at quotes,inside,inside,inside,at quotes
mid,0.025,0.225,0.35,0.135,1.725


## Results Set Generation

In [10]:
LUT = {
    "Trade_Size(ex)->Quote(Best)->Depth(Best)->Quote(Ex)->Depth(Ex)->Rev_Tick(All)": "\gls{GBM}",
    "(Ex)": " (Ex)",
    "(Best)": " (Best)",
    "(Classical)": " (Classical)",
    "(Classical-Size)": " (Classical, Size)",
    "Rev_": "Rev. ",
    "Trade_Size": "Trade Size",
    "Depth": "Depth",
    "->": " $\\to$ ",
    "Lr": "\gls{LR}",
    "Emo": "\gls{EMO}",
    "Clnv": "\gls{CLNV}",
    "OPTION_TYPE": "Option Type",
    # "(": "(",  # put interval start in math env
    # "]": "]",  # put interval end in math env
    "_": "$\_",
    "Gbm": "\gls{GBM}",
}

LUT_INDEX = {
    "OPTION_TYPE": "Option Type",
    "issue_type": "Security Type",
    "TRADE_SIZE_binned": "Trade Size",
    "year_binned": "Year",
    "ttm_binned": "Time to Maturity",
    "myn_binned": "Moneyness",
    "prox_q_binned": "Proximity to Quote",
}


def cell_str(x):
    x = x.title()
    for orig, sub in LUT.items():
        x = x.replace(orig, sub)
    # title-case everything
    return x


def highlight_max(s, props=""):
    return np.where(s == np.nanmax(s.values), props, "")


In [11]:
def set_tex_style(styler, caption, label, bold_axis=1):
    res = styler.set_caption(caption)
    res = (res.apply(
            highlight_max, props="font-weight:bold;", axis=bold_axis
        )
        .format(precision=4, decimal=".", thousands=",", escape=False, hyperlinks=None)
        .format_index(cell_str, axis=0)
        .format_index(cell_str, axis=1)
        # .hide(axis="index", level=0)
        .to_latex(
            f"{label}.tex",
            siunitx=True,
            position_float="centering",
            hrules=True,
            clines="skip-last;data",
            label="tab:" + label,
            caption=caption,
            convert_css=True,
        )
    )
    return res


In [12]:
classifiers = results_data.columns.tolist()
criterions = list(LUT_INDEX)


In [13]:
results_data.columns.tolist()


[('gbm', 'gbm(classical)'),
 ('gbm', 'gbm(classical-size)'),
 ('gbm', 'gbm(ml)'),
 ('classical', 'tick(ex)'),
 ('classical', 'quote(ex)'),
 ('classical', 'lr(ex)'),
 ('classical', 'emo(ex)'),
 ('classical', 'clnv(ex)'),
 ('classical',
  'trade_size(ex)->quote(best)->depth(best)->quote(ex)->depth(ex)->rev_tick(all)')]

In [14]:
X_print.head()

,buy_sell,TRADE_PRICE,OPTION_TYPE,issue_type,TRADE_SIZE_binned,year_binned,ttm_binned,myn_binned,prox_q_binned,mid,"(gbm, gbm(classical))","(gbm, gbm(classical-size))","(gbm, gbm(ml))","(classical, tick(ex))","(classical, quote(ex))","(classical, lr(ex))","(classical, emo(ex))","(classical, clnv(ex))","(classical, trade_size(ex)->quote(best)->depth(best)->quote(ex)->depth(ex)->rev_tick(all))"
index,,,,,,,,,,,,,,,,,,,
29930521,1,0.02,C,Others,"(0,1]",2016,<= 1,(0.7-0.9],at quotes,0.025,1,1,1,1.0,-1.0,-1.0,-1.0,-1.0,1.0
29894717,1,0.27,P,Stock options,"(5,11]",2016,<= 1,(0.9-1.1],inside,0.225,1,1,1,-1.0,1.0,1.0,1.0,1.0,1.0
29909660,1,0.20,C,Stock options,"(0,1]",2016,(1-2],(0.7-0.9],inside,0.350,1,1,1,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
29939322,-1,0.12,C,Others,"(0,1]",2016,<= 1,(0.9-1.1],inside,0.135,-1,-1,-1,-1.0,-1.0,-1.0,1.0,-1.0,-1.0
29883818,1,1.85,C,Stock options,"(3,5]",2016,(1-2],(0.9-1.1],at quotes,1.725,-1,1,1,-1.0,1.0,1.0,1.0,1.0,1.0


In [15]:
X_print[X_print["prox_q_binned"]=="unknown"]

,buy_sell,TRADE_PRICE,OPTION_TYPE,issue_type,TRADE_SIZE_binned,year_binned,ttm_binned,myn_binned,prox_q_binned,mid,"(gbm, gbm(classical))","(gbm, gbm(classical-size))","(gbm, gbm(ml))","(classical, tick(ex))","(classical, quote(ex))","(classical, lr(ex))","(classical, emo(ex))","(classical, clnv(ex))","(classical, trade_size(ex)->quote(best)->depth(best)->quote(ex)->depth(ex)->rev_tick(all))"
index,,,,,,,,,,,,,,,,,,,
29939396,-1,3.550000,P,Stock options,>11,2016,(3-6],(0.7-0.9],unknown,NaN,-1,-1,-1,1.0,-1.0,-1.0,-1.0,-1.0,-1.0
29655576,-1,291.799988,P,Index option,"(0,1]",2016,(3-6],(0.9-1.1],unknown,NaN,1,1,-1,1.0,1.0,1.0,1.0,1.0,1.0
29934780,-1,3.650000,C,Others,"(0,1]",2016,<= 1,(0.9-1.1],unknown,NaN,-1,-1,-1,1.0,1.0,1.0,1.0,1.0,-1.0
29937376,-1,1.600000,P,Others,>11,2016,<= 1,(0.9-1.1],unknown,NaN,1,-1,-1,1.0,-1.0,1.0,1.0,1.0,-1.0
29939363,1,1.950000,C,Stock options,>11,2016,> 12,(0.7-0.9],unknown,NaN,-1,-1,-1,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
37136668,1,0.050000,C,Stock options,"(1,3]",2017,<= 1,(0.7-0.9],unknown,NaN,-1,-1,-1,-1.0,1.0,-1.0,-1.0,-1.0,-1.0
37108888,1,2.120000,P,Stock options,"(3,5]",2017,<= 1,(0.9-1.1],unknown,2.040000,1,1,1,-1.0,1.0,1.0,-1.0,-1.0,1.0
37139864,1,7.300000,P,Others,"(0,1]",2017,(3-6],(0.9-1.1],unknown,NaN,1,1,1,1.0,-1.0,1.0,1.0,1.0,1.0


## Accuracy Calculation

In [ ]:
# FIXME: Find better approach
warnings.filterwarnings("ignore", category=np.VisibleDeprecationWarning)

result_dfs = []

for criterion in tqdm(criterions):
    results = []
    for classifier in tqdm(classifiers):
        acc_tot = accuracy_score(
            X_print["buy_sell"].astype("int8"), X_print[classifier]
        )

        res = (
            X_print.groupby([criterion])[["buy_sell", classifier]]
            .apply(
                lambda x: accuracy_score(x["buy_sell"].astype("int8"), x[classifier])
            )
            .mul(100)
            .rename(classifier)
        )
        res.loc["all"] = acc_tot * 100

        res.index.name = LUT_INDEX.get(criterion)
        results.append(res)

    # save aggregated results
    result_df = pd.concat(results, axis=1).T
    result_df.style.pipe(
        set_tex_style,
        caption=(f"long-tbd", "short-tbd"),
        label=f"{key.lower()}-{criterion.lower()}",
    )

    # store all result sets for later use
    result_dfs.append(result_df)


 56%|█████▌    | 5/9 [00:11<00:09,  2.43s/it]

In [ ]:
master = pd.concat(result_dfs, axis=1, keys=list(LUT_INDEX.values())).T


In [ ]:
master


In [ ]:
master.style.pipe(
    set_tex_style, caption=("master-long", "master-short"), label=f"{key}-master", bold_axis=0
)


## Effective Spread 🚧

In [ ]:
classifiers


In [ ]:
eff_dfs = []


def stats(x, classifier):

    nom = effective_spread(x[classifier], x["TRADE_PRICE"], x["mid"], mode="nominal")
    rel = (
        effective_spread(x[classifier], x["TRADE_PRICE"], x["mid"], mode="relative")
        * 100
    )

    # eff_spread_pred = effective_spread(x[classifier], x["TRADE_PRICE"], x["mid"], mode="none")
    # eff_spread_true = effective_spread(x["buy_sell"], x["TRADE_PRICE"], x["mid"], mode="none")
    # wilcoxon_res  = wilcoxon(eff_spread_pred, eff_spread_true, nan_policy="omit", zero_method="zsplit")

    return pd.Series(
        {
            "nominal": nom,
            "rel": rel,
            # 'statistic':wilcoxon_res.statistic,
            # 'pvalue':wilcoxon_res.pvalue
        }
    )


for criterion in tqdm(criterions):
    results = []

    for classifier in tqdm(classifiers):
        res = X_print.groupby([criterion])[
            ["TRADE_PRICE", "mid", classifier, "buy_sell"]
        ].apply(stats, classifier)
        results.append(res)

    # save aggregated results
    result_df = pd.concat(results, axis=1, keys=classifiers).T
    result_df.style.pipe(
        set_tex_style,
        caption=(f"long-tbd", "short-tbd"),
        label=f"{key.lower()}-{criterion.lower()}-eff-spread",
    )

    # store all result sets for later use
    eff_dfs.append(result_df)


In [ ]:
master_eff_spread = pd.concat(eff_dfs, axis=1, keys=LUT_INDEX.values()).T


In [ ]:
master_eff_spread


In [ ]:
master_eff_spread.style.pipe(
    set_tex_style,
    caption=("master-short", "master-long"),
    label=f"{key}-master-eff-spread",
    bold_axis=0,
)


## Change in Parenthesis 🅾️

```latex
# https://tex.stackexchange.com/questions/430283/table-with-numbers-in-parentheses-in-siunitx/430290#430290
\begin{table}
    \centering
    \caption{test of combination with change}
    \label{tab:combo}
    \begin{tabular}{lSSSSSSSS}
        \toprule
        {} & \multicolumn{2}{l}{Index option} & \multicolumn{2}{l}{Others} & \multicolumn{2}{l}{Stock options} & \multicolumn{2}{l}{all} \\
        \midrule
        classical-size & 1.0 & \parl-56.42\parr & 2.0 & \parl-74.35\parr & -73.5 & \parl-143.93\parr & 5.0 & \parl-67.33\parr \\
        \bottomrule
        \end{tabular}
\end{table}
```

In [ ]:
master

In [ ]:
base = master[[("classical","quote(ex)"),("classical", "trade_size(ex)->quote(best)->depth(best)->quote(ex)->depth(ex)->rev_tick(all)"), ("classical", "trade_size(ex)->quote(best)->depth(best)->quote(ex)->depth(ex)->rev_tick(all)")]]
revised = master[[("gbm","gbm(classical)"),("gbm","gbm(classical-size)"), ("gbm", "gbm(ml)")]]

In [ ]:

def combine_results(revised: pd.DataFrame, base: pd.DataFrame) -> pd.DataFrame:
    """
    Generate print layout like in Grauer et al.

    https://tex.stackexchange.com/questions/430283/table-with-numbers-in-parentheses-in-siunitx/430290#430290

    # see p. https://texdoc.org/serve/siunitx/0
    """
    # first, second layer of colum index
    c_1 = revised.columns.get_level_values(1)
    c_2 = ["nom"]
    midx = pd.MultiIndex.from_product([c_1, c_2])

    # copy data from revised add as (column, "nom")
    combo = pd.DataFrame(revised.values, index=revised.index, columns=midx)

    
    
    for i, mul_col in enumerate(combo.columns):

        combo[(mul_col[0], "pm")] = (
            "\parl" + (combo[mul_col] - base.iloc[:,i]).round(2).astype(str) + "\parr"
        )
        # print(combo.shape)
        # # define custom brackets that are not parsed by sunitx
        # combo.iloc[:,[(mul_col[0], "pm")]] = 
        # # sort to group together columns
        # combo.sort_index(axis=1, inplace=True)
        # cols.append(diff)
        combo.sort_index(axis=1, inplace=True)
        
    return combo


In [ ]:
diff = combine_results(revised, base)

diff.style.to_latex(
            f"diff-{key}.tex",
            siunitx=True,
            position_float="centering",
            hrules=True,
            clines="skip-last;data",
            label=f"tab:diff-{key}",
            caption=(f"long-diff-{key}", f"short-diff-{key}"),
            convert_css=True,
        )

In [ ]:
diff
